In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from Lattice_Class import Lattice, Observables
import numpy as np
import json
import os

# -----------------------------
# Set parameters
# -----------------------------
size = 50
iteration = 10000
sampling = 10
thermalisation = 1000
algorithm = "kawasaki"
error_analysis = "jackknife"
direction = "heating"
T_max = 3.0
T_min = 1.0
T_step = 0.1

# Output directory
output_dir = f"{algorithm}_{size}_{direction}"
os.makedirs(output_dir, exist_ok=True)

# Temperature values
if direction == 'cooling':
    T_vals = np.arange(T_max, T_min - T_step, -T_step)

elif direction == 'heating':
    T_vals = np.arange(T_min, T_max + T_step, T_step)

# -----------------------------
# Lists to store results
# -----------------------------
mag_list, mag_list_err = [], []
E_list, E_list_err = [], []
susceptibility_list, susceptibility_list_err = [], []
heat_capacity_list, heat_capacity_list_err = [], []

# -----------------------------
# Function to update lattice at each temperature
# -----------------------------
def update(T, old_grid, thermalisation):
    ising = Lattice(size=size, J=1, T=T, iterations=iteration,
                    algorithm=algorithm, thermalisation=thermalisation,
                    sampling=sampling, start_config=old_grid)
    
    list(ising.sim())  # Run the simulation
    new_grid = ising.grid

    # Compute observables
    Measurements = Observables(ising)
    Measurements.observables_with_errors(function=error_analysis)

    # Store results
    mag_list.append(Measurements.avg_mag)
    mag_list_err.append(Measurements.mag_error)
    E_list.append(Measurements.avg_E)
    E_list_err.append(Measurements.E_error)
    susceptibility_list.append(Measurements.susceptibility)
    susceptibility_list_err.append(Measurements.susceptibility_error)
    heat_capacity_list.append(Measurements.heat_capacity)
    heat_capacity_list_err.append(Measurements.heat_capacity_error)

    # Save individual temperature JSON
    data = {
        "T": float(T),
        "magnetisation": [float(x) for x in ising.magnetisation],
        "total_energy": [float(x) for x in ising.totenergy]
    }
    filename = os.path.join(output_dir, f"T_{T:.3f}.json")
    with open(filename, "w") as f:
        json.dump(data, f, indent=4)

    return new_grid

# -----------------------------
# Run simulation over temperatures and initialize with grid type
# -----------------------------
if algorithm == 'kawasaki':
    old_grid = np.random.choice([-1, 1], size=(size, size)) # For kawasaki, always start with a random configuration

elif algorithm == 'glauber':
    if direction == 'cooling':
        old_grid = np.random.choice([-1, 1], size=(size, size)) # random initial configuration for cooling
    elif direction == 'heating':
        old_grid = np.ones((size, size), dtype=int) # ordered spin configuration up for heating


old_grid = update(T_vals[0], old_grid, thermalisation * 5) # Extra thermalisation for first T
for T in T_vals[1:]:
    old_grid = update(T, old_grid, thermalisation)


# -----------------------------
# Save all observables in one JSON
# -----------------------------
results = {
    "T_vals": T_vals.tolist(),
    "parameters": {
        "size": size,
        "J": 1,
        "iterations": iteration,
        "algorithm": algorithm,
        "thermalisation": thermalisation,
        "sampling": sampling,
        "error_analysis": error_analysis
    },
    "magnetization": {"values": mag_list, "errors": mag_list_err},
    "energy": {"values": E_list, "errors": E_list_err},
    "susceptibility": {"values": susceptibility_list, "errors": susceptibility_list_err},
    "heat_capacity": {"values": heat_capacity_list, "errors": heat_capacity_list_err}
}

filename = os.path.join(output_dir, "Observables.json")
with open(filename, "w") as f:
    json.dump(results, f, indent=4)

print(f"Results saved to {filename}")

plt.figure(figsize=(14, 10))

# Average Magnetization per spin
plt.subplot(2, 2, 1)
plt.errorbar(T_vals, mag_list, yerr=mag_list_err, fmt='o', markersize=2, elinewidth=2, capsize=3)
plt.title(r"Magnetization $M$", fontsize=10)
plt.xlabel(r"Temperature $k_bT$", fontsize=9)
plt.ylabel(r"$\langle |M| \rangle$", fontsize=9)

# Average Energy per spin
plt.subplot(2, 2, 2)
plt.errorbar(T_vals, E_list, yerr=E_list_err, fmt='s', markersize=2,
             elinewidth=2, capsize=3)
plt.title(r"Energy $E$", fontsize=10)
plt.xlabel(r"Temperature $k_bT$", fontsize=9)
plt.ylabel(r"$\langle E \rangle$", fontsize=9)

# Susceptibility
plt.subplot(2, 2, 3)
plt.errorbar(T_vals, susceptibility_list, yerr=susceptibility_list_err, fmt='^', markersize=2,
             elinewidth=2, capsize=3)
plt.title(r"Magnetic Susceptibility $\chi$", fontsize=10)
plt.xlabel(r"Temperature $k_bT$", fontsize=9)
plt.ylabel(r"$\chi$", fontsize=9)

# Heat Capacity
plt.subplot(2, 2, 4)
plt.errorbar(T_vals, heat_capacity_list, yerr=heat_capacity_list_err, fmt='d', markersize=2,
             elinewidth=2, capsize=3)
plt.title(r"Heat Capacity $C_v$", fontsize=10)
plt.xlabel(r"Temperature $k_bT$", fontsize=9)
plt.ylabel(r"$C_v$", fontsize=9)

plt.tight_layout()
plt.show()

Average Magnetization: 16.0 Average Energy: -4707.120879120879 Susceptibility: 0.0 Heat Capacity: 8.962938810640573
Average Magnetization: 16.0 Average Energy: -4722.373626373626 Susceptibility: 0.0 Heat Capacity: 0.0690483941642706
Average Magnetization: 16.0 Average Energy: -4694.613386613387 Susceptibility: 0.0 Heat Capacity: 0.14269602081738408
